For faster training with a lightweight model, you can use DistilBERT, which is a smaller, faster, and lighter version of BERT. DistilBERT retains 97% of BERT's language understanding while being 60% faster and 40% smaller. Here's how you can fine-tune DistilBERT using the same dataset:

**Install libraries**

In [ ]:
!pip install pandas transformers==4.28.0 scikit-learn datasets accelerate


**Create dataset**

In [2]:

import pandas as pd

# Define labels and sample messages
harm_messages = [
    "I'm going to hurt you.", "You should be scared.", "I'm going to ruin your life.",
    "You will regret this.", "I will find you.", "You won't be safe.",
    "I hate you.", "I'm going to make you suffer.", "You deserve pain.", "I'm going to end you.",
    # ... add more harm messages to total 200
]

spam_messages = [
    "Congratulations! You've won a free iPhone!", "You've been selected for a prize!",
    "Claim your $1000 gift card now.", "Act now and get a free trip!", "You've won a lottery!",
    "Click here for a free vacation!", "Limited time offer! Free access to premium content.",
    "Get rich quick! Learn the secrets of millionaires.", "Exclusive deal just for you!",
    "Earn $5000 per week from home.", "Your account has been compromised.",
    # ... add more spam messages to total 200
]

generic_messages = [
    "How's your day going?", "What are you doing this weekend?", "Can you help me with this task?",
    "I'm looking forward to our meeting.", "Let's catch up soon.", "Happy birthday!",
    "Have you seen the latest movie?", "What's your favorite book?", "Where do you want to travel next?",
    "How was your vacation?", "I'm working on a new project.",
    # ... add more generic messages to total 200
]

# Ensuring all lists have 200 messages
while len(harm_messages) < 200:
    harm_messages.append(harm_messages[len(harm_messages) % 10])
while len(spam_messages) < 200:
    spam_messages.append(spam_messages[len(spam_messages) % 11])
while len(generic_messages) < 200:
    generic_messages.append(generic_messages[len(generic_messages) % 11])

# Create a DataFrame
data = {
    "label": ["harm"]*200 + ["spam"]*200 + ["generic"]*200,
    "message": harm_messages + spam_messages + generic_messages
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('labeled_messages.csv', index=False)



In [3]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load the dataset
df = pd.read_csv('labeled_messages.csv')

# Preprocess the data
df['label'] = df['label'].map({'harm': 0, 'spam': 1, 'generic': 2})
train_df, eval_df = train_test_split(df, test_size=0.1)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Add padding token
#tokenizer.pad_token = tokenizer.eos_token
#model.config.pad_token_id = tokenizer.pad_token_id
def tokenize_function(examples):
    return tokenizer(examples['message'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained('./anil-llm')
tokenizer.save_pretrained('./anil-llm')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.we

Map:   0%|          | 0/540 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


('./anil-llm/tokenizer_config.json',
 './anil-llm/special_tokens_map.json',
 './anil-llm/vocab.txt',
 './anil-llm/added_tokens.json')

In [4]:
from transformers import pipeline

# Load the fine-tuned model
model_path = './anil-llm'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Create a pipeline for text classification
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# New sample messages
new_messages = [
    "I am going to harm you.",
    "Congratulations! You've won a free car!",
    "How are you today?"
]

# Classify the new messages
for message in new_messages:
    result = classifier(message)
    print(f"Message: {message}\nLabel: {result[0]['label']}\n")


Message: I am going to harm you.
Label: LABEL_0

Message: Congratulations! You've won a free car!
Label: LABEL_1

Message: How are you today?
Label: LABEL_2



#({'harm': 0, 'spam': 1, 'generic': 2})